In [ ]:
import tensorflow as tf
import os
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
!aws s3 cp s3://sagemaker-mumbai-data-set data_set --recursive

In [ ]:
!ls data_set/ena_data/

In [ ]:
IMG_SIZE = 160
BATCH_SIZE = 64

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

INPUT_TENSOR_NAME = "inputs_input"

In [ ]:
    base_model = InceptionV3(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
    base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(units=256, activation='relu'))
    model.add(layers.Dense(units=5, activation='softmax'))

In [ ]:
tf.keras.models.save_model(
    model,
    '/opt/',
    overwrite=True,
    save_format='tf'
)

In [ ]:
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [ ]:
model.input

In [ ]:
model.model

In [ ]:
def keras_model_fn(hyperparameters):
    base_model = InceptionV3(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
    base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(units=256, activation='relu'))
    model.add(layers.Dense(units=5, activation='softmax'))
    
    adamOptimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(optimizer=adamOptimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )
    return model

In [ ]:
model = keras_model_fn(hyperparameters=0)

In [ ]:
len(model.trainable_weights)

In [ ]:
!tensorboard --logdir s3://quikr-datascience-mumbai/image_classify/ena_classify/sagemaker_output/ --host localhost --port 6006

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
generator = datagen.flow_from_directory('data_set/ena_data/train/', target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE)

In [ ]:
list(generator.class_indices.keys())

In [ ]:
generator.n // generator.batch_size

In [ ]:
math.ceil(12600/64)

In [ ]:
validation_generator = datagen.flow_from_directory(
        'data_set/ena_data/validation', target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE)

In [ ]:
model.fit_generator(
        generator,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=800)